In [1]:
try:
    if is_init:
        pass
except:
    %pwd
    %cd ..
    %pwd
    is_init = True

/home/spencersong/metamaterials_ai/src


In [2]:
from pathlib import Path
from src.utils import FileUtils
from config import Config
import torch
import numpy as np
import random
import itertools
import pandas as pd
from src.models import DirectModel
from src.models import InverseModel
from typing import List, Optional, Tuple
from src.predictor import Predictor
from scipy.interpolate import interp1d
from utils import rmse
data_folder = '../local_data'
work_folder = '../local_work'


# Ensemble!!

In [9]:
inc_params = torch.load(
                Path(f"{data_folder}/inconel-revised-raw-shuffled.pt"))["laser_params"]
steel_params = torch.load(
                Path(f"{data_folder}/stainless-steel-revised-shuffled.pt"))["laser_params"] #stainless-steel-revised-shuffled inconel-revised-raw-shuffled
max_speed, max_spacing = steel_params.max(0)[0][0].item(), steel_params.max(0)[0][1].item()
min_speed, min_spacing = steel_params.min(0)[0][0].item(), steel_params.min(0)[0][1].item()

In [4]:
# def denormalize_decode_result(y_hat, max_speed, max_spacing, min_speed, min_spacing):
#     """input: 1,14 tensor
#         output: 1,3 tensor with wattage no longer one hot encoded"""
#     watts = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3]

#     watt_arg = torch.argmax(y_hat[0][2:])
#     y_final = torch.empty(3, dtype=torch.float32)
#     (laser_params - laser_params.min(0).values) / (
#         laser_params.max(0).values - laser_params.min(0).values
#     )
#     y_final[0] = ((max_speed - min_speed) * y_hat[0][0]) + min_speed # TODO call the scale
#     y_final[1] = ((max_spacing-min_spacing) * y_hat[0][1]) + min_spacing # TODO call the scale
#     y_final[2]= watts[watt_arg]
#     return y_final

In [10]:
def interpolate(csv):
    num_wavelens = 800
    emiss_plot = [item[1][1] for item in read_file.iterrows() if (item[1][1] != 1.0 and item[1][0] < 12)]
    wavelength_plot = [item[1][0] for item in read_file.iterrows() if (item[1][1] != 1.0 and item[1][0] < 12)]

    # Reverse to sort in ascending rather than descending order.
    # emiss_plot.reverse()
    # wavelength_plot.reverse()

    interp_wavelen = np.linspace(
                min(wavelength_plot), max(wavelength_plot), num=num_wavelens
            )
    interp_emiss = interp1d(wavelength_plot, emiss_plot)(interp_wavelen).astype(np.float32).reshape(1, 800)
    predictor_input = torch.from_numpy(interp_emiss)
    return predictor_input

In [11]:
# read_file
read_file = pd.read_csv('../src/notebooks/sample_input.csv')
desired_emiss = interpolate(read_file) #torch.from_numpy(np.array(torch.randn(1, 800) * 3))
include_inconel = False #Change ME
include_stainless = True #Change ME

In [12]:
predictor = Predictor(desired_emiss, True, True)
y_hat_ss, y_hat_inc = predictor.run_inverse(predictor.desired)
direct_ss_emiss_y_hat, direct_inc_emiss_y_hat = predictor.run_direct(y_hat_ss, y_hat_inc)
best_substrate, best_params, best_rmse = predictor.best_predictor(True, True, direct_ss_emiss_y_hat, direct_inc_emiss_y_hat, predictor.desired)

/home/spencersong/metamaterials_ai/src


/home/spencersong/anaconda3/envs/meta/lib/python3.10/site-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '
/home/spencersong/anaconda3/envs/meta/lib/python3.10/site-packages/pytorch_lightning/core/saving.py:217: UserWarning: Found keys that are not in the model state dict but in the checkpoint: ['direct_model.model.0.weight', 'direct_model.model.0.bias', 'direct_model.model.2.layers.0.block.0.weight', 'direct_model.model.2.layers.0.block.0.bias', 'direct_model.model.2.layers.1.block.0.weight', 'direct_model.model.2.layers.1.block.0.bias', 'direct_model.model.2.layers.2.block.0.weight', 'direct_model.model.2.layers.2.block.0.bias', 'direct_model.model.2.layers.3.block.0.weight', 'direct_model.model.2.layers.3.block.0.bias', 'direct_model.model.2.layers.4.block.0.weight', 'direct_model.model.2.

In [13]:
#outputs of ensemble!!
print(best_substrate, best_params, best_rmse)

stainless [577.7000122070312, 34.29999923706055, 0.20000000298023224] 0.05857977271080017


# Inverse Model Only


In [52]:
x = desired_emiss

if include_inconel:
    inverse_ss_filepath = Path(f"{work_folder}/saved_best/I-0.9-res-ann-stainless.ckpt") #CHANGEME
    print(inverse_ss_filepath)
    if not Path.is_file(inverse_ss_filepath):
        raise Exception(f'Model file does not exist at {inverse_ss_filepath}!')

    i_model_ss = InverseModel.load_from_checkpoint(inverse_ss_filepath, strict=False)
    i_model_ss.eval()
    with torch.no_grad():
        y_hat_ss = i_model_ss(x)
    #y_hat_ss_clean = denormalize_decode_result(y_hat_ss, max_speed, max_spacing, min_speed, min_spacing)

if include_stainless:
    inverse_inc_filepath = Path(f"{work_folder}/saved_best/I-0.9-res-ann-inconel.ckpt") #CHANGEME
    print(inverse_inc_filepath)
    if not Path.is_file(inverse_inc_filepath):
        raise Exception(f'Model file does not exist at {inverse_inc_filepath}!')

    i_model_inc = InverseModel.load_from_checkpoint(inverse_inc_filepath, strict=False)
    i_model_inc.eval()
    with torch.no_grad():
        y_hat_inc = i_model_inc(x)
    #y_hat_inc_clean = denormalize_decode_result(y_hat_inc, max_speed, max_spacing, min_speed, min_spacing)

../local_work/saved_best/I-0.9-res-ann-stainless.ckpt


/home/spencersong/anaconda3/envs/meta/lib/python3.10/site-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '
/home/spencersong/anaconda3/envs/meta/lib/python3.10/site-packages/pytorch_lightning/core/saving.py:217: UserWarning: Found keys that are not in the model state dict but in the checkpoint: ['direct_model.model.0.weight', 'direct_model.model.0.bias', 'direct_model.model.2.layers.0.block.0.weight', 'direct_model.model.2.layers.0.block.0.bias', 'direct_model.model.2.layers.1.block.0.weight', 'direct_model.model.2.layers.1.block.0.bias', 'direct_model.model.2.layers.2.block.0.weight', 'direct_model.model.2.layers.2.block.0.bias', 'direct_model.model.2.layers.3.block.0.weight', 'direct_model.model.2.layers.3.block.0.bias', 'direct_model.model.2.layers.4.block.0.weight', 'direct_model.model.2.

In [53]:
# y_final2 = onehot_to_continuous2(y_hat)
#print(denormalize_decode_result(y_hat,max_speed, max_spacing, min_speed, min_spacing))

#if you want hardcoded max, min values instead
#print(denormalize_decode_result(y_hat,700.0, 42.0, 10.0, 1.0))

# Validate Ensemble with Direct Model


In [54]:
from src.models import DirectModel

direct_ss_filepath = Path(""
    f"{work_folder}/saved_best/D-0.9-res-ann-stainless.ckpt") #CHANGEME
print(direct_ss_filepath)
if not Path.is_file(direct_ss_filepath):
    raise Exception(f'Model file does not exist at {direct_ss_filepath}!')

direct_inc_filepath = Path(""
    f"{work_folder}/saved_best/D-0.9-res-ann-inconel.ckpt") #CHANGEME
print(direct_inc_filepath)
if not Path.is_file(direct_inc_filepath):
    raise Exception(f'Model file does not exist at {direct_inc_filepath}!')

../local_work/saved_best/D-0.9-res-ann-stainless.ckpt
../local_work/saved_best/D-0.9-res-ann-inconel.ckpt


In [55]:
if include_stainless:
    d_ss_model = DirectModel.load_from_checkpoint(direct_ss_filepath)
    d_ss_model.eval()
    # x = torch.randn(1, 14)
    # intersect_preds = []
    with torch.no_grad():
        direct_ss_emiss_y_hat = d_ss_model(y_hat_ss.reshape(1,14))
        
if include_inconel:
    d_inc_model = DirectModel.load_from_checkpoint(direct_inc_filepath)
    d_inc_model.eval()
    # x = torch.randn(1, 14)
    # intersect_preds = []
    with torch.no_grad():
        direct_inc_emiss_y_hat = d_inc_model(y_hat_ss.reshape(1,14))

### Find Best Output

In [66]:
substrates = ['inconel', 'stainless']
best_params = []
best_rmse = 0
if include_inconel and include_stainless:
    if rmse(direct_inc_emiss_y_hat, desired_emiss) < rmse(direct_ss_emiss_y_hat, desired_emiss):
        best_substrate = substrates[0]
        best_params = denormalize_decode_result(y_hat_inc, max_speed, max_spacing, min_speed, min_spacing)
        best_rmse = rmse(direct_inc_emiss_y_hat, desired_emiss).item()
    else:
        best_substrate = substrates[1]
        best_params = denormalize_decode_result(y_hat_ss, max_speed, max_spacing, min_speed, min_spacing)
        best_rmse = rmse(direct_ss_emiss_y_hat, desired_emiss).item()
elif include_inconel:
    best_substrate = substrates[0]
    best_params = denormalize_decode_result(y_hat_inc, max_speed, max_spacing, min_speed, min_spacing)
    best_rmse = rmse(direct_inc_emiss_y_hat, desired_emiss).item()
else:
    best_substrate = substrates[1]
    best_params = denormalize_decode_result(y_hat_ss, max_speed, max_spacing, min_speed, min_spacing)
    best_rmse = rmse(direct_ss_emiss_y_hat, desired_emiss).item()

best_params = [torch.round(best_params[0], decimals = 1).item(), torch.round(best_params[1], decimals = 1).item(), torch.round(best_params[2], decimals = 1).item()]

In [67]:
print(best_substrate)
print(best_params)
print(best_rmse)

inconel
[91.69999694824219, 0.20000000298023224, 0.4000000059604645]
3.0110976696014404


In [100]:
params = torch.load(
                Path(f"{data_folder}/inconel-revised-raw-shuffled.pt"))["laser_params"] #stainless-steel-revised-shuffled inconel-revised-raw-shuffled
max_speed, max_spacing = params.max(0)[0][0].item(), params.max(0)[0][1].item()
min_speed, min_spacing = params.min(0)[0][0].item(), params.min(0)[0][1].item()
decoded_preds = [denormalize_decode_result(pred, max_speed, max_spacing, min_speed, min_spacing) for pred in param_preds]
for r in decoded_preds:
    r[0]=torch.round(r[0], decimals = -1)
    r[1]=torch.round(r[1], decimals = 0)

In [101]:
param_space = [np.arange(min_speed, max_speed + 10, 10).tolist(), np.arange(min_spacing, max_spacing + 1, 1).tolist(), np.around(np.arange(.2, 1.4, .1), decimals = 1).tolist()]
#get set of all possible parameter combinations
param_all_comb = set(list(itertools.product(*param_space)))
wattages = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3]
space_params = set([tuple(np.around(x.tolist(), decimals= 1)) for x in decoded_preds]) #29988
        #find the combination parameters NOT in the training space
overlaps = list(param_all_comb.intersection(space_params)) #len() = 5292
unused_params = list(set(overlaps).symmetric_difference(space_params)) #len() = 5292
#convert preds to set
print(len(space_params))
print(len(overlaps)) #332
len(unused_params) #216



591
352


239

In [63]:
#send these to minok
count_samples_to_minok = 196
unused_params[:count_samples_to_minok]#overlaps)
#overlaps[:195]


[(570.0, 13.0, 1.2),
 (330.0, 13.0, 1.2),
 (610.0, 14.0, 1.2),
 (340.0, 10.0, 1.2),
 (550.0, 11.0, 1.2),
 (510.0, 9.0, 1.2),
 (500.0, 7.0, 1.2),
 (660.0, 9.0, 1.2),
 (370.0, 12.0, 1.2),
 (440.0, 9.0, 1.2),
 (650.0, 10.0, 1.2),
 (490.0, 13.0, 1.2),
 (220.0, 12.0, 1.2),
 (610.0, 13.0, 1.2),
 (420.0, 7.0, 1.2),
 (180.0, 10.0, 1.2),
 (640.0, 8.0, 1.2),
 (470.0, 11.0, 1.2),
 (460.0, 8.0, 1.2),
 (340.0, 13.0, 1.2),
 (510.0, 12.0, 1.2),
 (580.0, 9.0, 1.2),
 (100.0, 10.0, 1.2),
 (280.0, 9.0, 1.2),
 (260.0, 11.0, 1.2),
 (450.0, 14.0, 1.2),
 (630.0, 13.0, 1.2),
 (440.0, 12.0, 1.2),
 (140.0, 12.0, 1.2),
 (420.0, 10.0, 1.2),
 (360.0, 10.0, 1.2),
 (670.0, 11.0, 0.5),
 (170.0, 8.0, 1.2),
 (100.0, 13.0, 1.2),
 (380.0, 9.0, 1.2),
 (520.0, 12.0, 1.2),
 (500.0, 10.0, 1.2),
 (160.0, 14.0, 1.2),
 (200.0, 10.0, 1.2),
 (550.0, 14.0, 1.2),
 (620.0, 11.0, 1.2),
 (130.0, 9.0, 1.2),
 (360.0, 13.0, 1.2),
 (480.0, 13.0, 1.2),
 (640.0, 11.0, 1.2),
 (0.0, 28.0, 0.8),
 (180.0, 13.0, 1.2),
 (600.0, 9.0, 1.2),
 (570.0

In [256]:
#intersecting parameters!!!!!!!
intersect_x = []
wattage_idxs = {
        0.2: 0,
        0.3: 1,
        0.4: 2,
        0.5: 3,
        0.6: 4,
        0.7: 5,
        0.8: 6,
        0.9: 7,
        1.0: 8,
        1.1: 9,
        1.2: 10,
        1.3: 11,
}      
for i in unused_params:
        i = list(i)
        holder = i[:2] + np.zeros(12).tolist()
        holder[wattage_idxs[i[2]] + 2] = 1.0
        intersect_x.append(holder)
output = torch.tensor(intersect_x)
output

tensor([[600.,  34.,   0.,  ...,   0.,   0.,   0.],
        [590.,  14.,   0.,  ...,   0.,   1.,   0.],
        [580.,  31.,   0.,  ...,   0.,   0.,   0.],
        ...,
        [640.,  16.,   0.,  ...,   0.,   1.,   0.],
        [520.,  38.,   0.,  ...,   0.,   0.,   0.],
        [300.,  37.,   0.,  ...,   0.,   0.,   0.]], dtype=torch.float64)

In [136]:
for i in range(len(params)):
    curr = params[i].flatten().tolist()
    if curr in output.tolist():
        print(i)

12058
19562
19609


In [149]:
from utils import rmse
# params[12058]
data = torch.load(
                Path(f"{data_folder}/stainless-steel-revised-shuffled.pt"))

intersect_train_emiss = []
intersect_train_emiss.append(data["emissivity"][12058])
intersect_train_emiss.append(data["emissivity"][19562])
intersect_train_emiss.append(data["emissivity"][19609])

# ONNX

In [ ]:
# import onnxruntime 
# from pathlib import Path
# import numpy as np
# import os

In [ ]:
ort_session = onnxruntime.InferenceSession(filepath)

In [ ]:
# filepath = Path(f"{work_folder}/saved_best/best_inference_direct.onnx")

# ort_session = onnxruntime.InferenceSession(filepath)
# input_name = ort_session.get_inputs()[0].name
# ort_inputs = {input_name: np.random.randn(2, 14)}
# ort_outs = ort_session.run(None, ort_inputs)
